In [ ]:
# Carga de las librerías
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocesamiento de los datos

Se va a realizar el preprocesado de los datos de expresión transcriptómica obtenidos a partir de microarrays Clariom D Human de Affymetrix, los cuales, previamente, han sido preprocesados mediante el método RMA. 

En particular, se parte de unos datos de expresión cuyo fondo ya ha sido corregido, han sido normalizados y resumidos. 

En cuanto al diseño experimental, se dipone de datos de expresión transcriptómica de lineas celulares tanto de cáncer de colon como de colonocitos normales. Además, de cada una de ellas, se tienen muestras tratadas y sin tratar con DFMO, un inhibidor suicida de la ODC (ornitin descarboxilasa).

## Lectura de datos

In [ ]:
# Lectura datos expresion
df_expresion = pd.read_csv("../data/matriz_expresion.csv", index_col=0)
# Lectura covariables
df_covariables = pd.read_csv("../data/matriz_covariables.csv", index_col=0)

In [ ]:

# Mostrar datos
print(f"A continuación, se muestran las primeras filas de la matriz de expresión procesada (de dimensiones {df_expresion.shape}):\n")
df_expresion.head()

In [ ]:
print(f"A continuación, se muestran las primeras filas de la matriz de covariables (de dimensiones {df_covariables.shape}):\n")
df_covariables.head()

## Valores Faltantes
En primer lugar, se comprueba si existen valores faltantes en el conjunto de datos. En caso de que existan, se procederá a su imputación o eliminación, dependiendo del porcentaje de datos faltantes.

In [ ]:
# Comprobación de valores faltantes en la matriz de expresión
n_faltantes_df_exp = df_expresion.isnull().sum().sum()
print(f"- En la matriz de expresión ('df_expresion') hay un total de {n_faltantes_df_exp} valores faltantes.")

# Comprobación de valores faltantes en la matriz de covariables
n_faltantes_df_cov = df_covariables.isnull().sum().sum()
print(f"- En la matriz de covariables ('df_covariables') hay un total de {n_faltantes_df_cov} valores faltantes.")

## Renombrado de muestras

Para que sean más fácilmente identificables las muestras, se renombran las columnas de la matriz de expresión con los valores de la columna 'SampleID' de la matriz de covariables:

In [ ]:
df_expresion.columns = df_covariables["SampleID"]
df_expresion

## Reordenamiento de las columnas

Se van a reordenar tanto las filas de la matriz de covariables como las columnas de la matriz de expresión para que sigan el mismo orden. En concreto, dicho orden será el siguiente: muestras de colonocitos normales sin tratar, muestras de colonocitos normales tratadas con DFMO, muestras de células de cáncer de colon sin tratar y muestras de células de cáncer de colon tratadas con DFMO.

In [ ]:
# Reordenamiento de las filas de la matriz de covariables
df_covariables = df_covariables.set_index("SampleID")
# Ordenar según la columna Línea celular y luego Tratamiento
df_covariables.sort_values(by = ["Linea", "Tratamiento"], inplace=True)
df_covariables

A continuación, se procede a reordenar las columnas de la matriz de expresión según el nuevo orden de las filas de la matriz de covariables:

In [ ]:
# Reordenar la matriz de expresión según el nuevo orden de las filas de la matriz de covariables
df_expresion = df_expresion[df_covariables.index]
df_expresion

## Renombrado de las filas

Los microarrays de Affymetrix incluyen varios sondas (probesets) para cada gen. Por tanto, es necesario resumir la expresión de los genes a partir de la expresión de los probesets. Para ello, se renombran las filas de la matriz de expresión con los valores de la columna 'Symbol' del archivo de anotación:

In [ ]:
# Carga del archivo de anotación
df_anotacion = pd.read_csv("../data/probe2symbol.csv")
df_anotacion = df_anotacion.dropna(subset = "SYMBOL")
df_anotacion = df_anotacion.iloc[:, 1:]
df_anotacion

En primer lugar, se hará un 'inner join' entre la matriz de expresión y la de anotación, de forma que se eliminen aquellos probesets que no tengan asociado un símbolo de gen. A continuación, se sustituye el índice (nombres de fila) por los límbolos de gen y se eliminan las columnas 'PROBEID' y 'SYMBOL', que ya no son necesarias.

In [ ]:
# Left join matriz de expresion con anotación mediante rownames=PROBEID
df_expr_symbol = df_expresion.merge(df_anotacion, left_index=True, right_on='PROBEID', how = 'inner')
df_expr_symbol.index = df_expr_symbol['SYMBOL']
df_expr_symbol = df_expr_symbol.drop(columns=['PROBEID', 'SYMBOL'])
df_expr_symbol = df_expr_symbol.sort_index()
df_expr_symbol

Cabe destacar que no todos los probesets tienen asociado un símbolo de gen, por lo que se pierden algunas filas en este proceso. 

In [ ]:
# Sondas que no tienen símbolo de gen asociado
n_sin_symbol = df_expresion.shape[0] - df_expr_symbol.shape[0]
print(f"En particular, se pierden {n_sin_symbol} sondas que no tienen símbolo de gen asociado.")

## Duplicados: Eliminar/Combinar filas con el mismo GENE SYMBOL

No es de extrañar que haya varios conjuntos de sondas (probesets) que midan la expresión del mismo gen, ya que un gen puede tener varias variantes de transcripción (transcriptos) y, por tanto, varios probesets pueden estar diseñados para medir la expresión de diferentes transcriptos del mismo gen:

In [ ]:
duplicados = df_expr_symbol.index[df_expr_symbol.index.duplicated()]
print(f"En concreto, hay un total de {len(duplicados)} 'GENE SYMBOL' duplicados")
df_anotacion["SYMBOL"].value_counts()[df_anotacion["SYMBOL"].value_counts() > 1]

Por ejemplo, en el caso del gen 'F10', hay 2 probesets diferentes que miden la expresión de este gen:

In [ ]:
df_anotacion.where(df_anotacion["SYMBOL"] == "F10").dropna()

Existen multitud de estrategias para abordar este problema: desde la combinación de probesets hasta la selección de un único probeset representativo por gen. En este caso, se opta por la primera opción. En cuanto a la combinación de probesets, existen varias alternativas, como la media, la mediana o la selección del probeset con mayor varianza. 

### Combinar probesets mediante la mediana

En este caso, se opta por calcular la mediana de expresión de los probesets que miden el mismo gen.

In [ ]:
# Combinar filas con el mismo GENE SYMBOL usando la mediana
df_expr_symbol_median = df_expr_symbol.groupby(df_expr_symbol.index).median()
df_expr_symbol_median

Por ejemplo, para el gen 'F10' tiene 2 probesets asociados:

In [ ]:
print("Se pasa de tener:")
print(df_expr_symbol.loc[["F10"], df_expr_symbol.columns[:5]])
print("\nA tener:")
# Se muestran las filas correspondientes al gen 'RNA5S4' antes de agrupar
print(df_expr_symbol_median.loc[["F10"], df_expr_symbol_median.columns[:5]])


Como resultado, se obtiene una nueva matriz de expresión en la que cada gen está representado por un único valor de expresión, que es la mediana de los valores de expresión de los probesets asociados a ese gen.

In [ ]:
print(f"Como resultado, Se pasa de una matriz de expresión de dimensiones {df_expresion.shape} a una matriz de dimensiones {df_expr_symbol_median.shape}.")

# Guardar la matriz de expresión final

In [ ]:
# Guardar la matriz de expresión final
df_expr_symbol_median.to_csv("../data/matriz_expr_symbol_median.csv")
# Guardar la matriz de covariables ordenada
df_covariables.to_csv("../data/matriz_covariables_ordenada.csv")